In [52]:
# Import libraries
import requests
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np

from tqdm import tqdm
import glob
import time
import json
import os

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [3]:
def getresponse(url):
    headers = {
        "User-Agent":"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36",
        "Accept-Language":"en-US,en;q=0.9,is;q=0.8,da;q=0.7",
        "Referer":"https://www.google.com/",
        "DNT":"1",
    }
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    return res

In [4]:
# get field names
url = "https://etfdb.com/etfs/"
response = getresponse(url)
soup = BeautifulSoup(response.text, 'lxml')

# field_list = soup.find("ul", attrs={"class":"nav nav-pills"})
# fields = [li.text.replace("New","").strip().replace(" ", "-").lower() for li in field_list.find_all("li")]
# fields 

In [73]:
regiontable = soup.find(id = "region-b1-table")
regions = pd.read_html(str(regiontable))[0].loc[:, ["ETF THEME","# OF ETFs"]]
regions.columns = ["region", "count"]

regions["region url"] = regions["region"].apply(lambda x: x.lower().replace(".", "").replace(" ", "-"))
regions

,region,count,region url
0,Africa,5,africa
1,Broad Asia,40,broad-asia
2,Developed Asia Pacific,80,developed-asia-pacific
3,Developed Europe,68,developed-europe
4,Developed Markets,456,developed-markets
5,Emerging Asia Pacific,106,emerging-asia-pacific
6,Emerging Europe,5,emerging-europe
7,Emerging Markets,108,emerging-markets
8,Frontier Markets,1,frontier-markets
9,Global,337,global


In [114]:


def download_regions(regionsdf):
    from Data.etfdbcredentials import username, password
    # open browser
    options = webdriver.ChromeOptions()
    # options.add_argument('--headless')
    service = Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=service, options=options)



    url = "https://etfdb.com/members/login/"
    browser.get(url)

    # find and fill in username field
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.ID, 'user_login'))
    ).send_keys(username)

    # find and fill inn password field
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.ID, 'password'))
    ).send_keys(password)

    # find login button and click
    WebDriverWait(browser, 10).until(
        EC.element_to_be_clickable((By.ID, 'login-button'))
    ).click()




    # wait a bit
    time.sleep(5)



    error_regions = []
    for region in tqdm(regionsdf["region url"]):
        region_pretty = list(regionsdf[regionsdf["region url"] == region]["region"])[0]
        
        try:
            url = f"https://etfdb.com/etfs/region/{region}"
            browser.get(url)

            # find download button and click
            WebDriverWait(browser, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//*[@id="etfs"]/tfoot/tr/td/a[2]'))
            ).click()
    
            
            # wait for file
            filelist = [file.split("/")[-1] for file in glob.glob("/Users/ThorvaldurIngi/Downloads/*.csv")]
            file_name = "etfs_details_type_fund_flow.csv"
            start_time = time.time()
            while not file_name in filelist:
                time.sleep(1)
                time_delta = time.time() - start_time
                if time_delta > 15:
                    error_regions.append(region_pretty)
                    break
                filelist = [file.split("/")[-1] for file in glob.glob("/Users/ThorvaldurIngi/Downloads/*.csv")]
             
            os.rename("/Users/ThorvaldurIngi/Downloads/"+file_name, f"/Users/ThorvaldurIngi/Downloads/{region_pretty}.csv")

        except:
            error_regions.append(region_pretty)

    browser.quit()
    return error_regions

In [115]:
download_regions(regions)

100%|███████████████████████████████████████████| 14/14 [01:04<00:00,  4.60s/it]


[]

In [127]:
filelist = glob.glob("Data/Regions files/*.csv")

In [128]:
filelist

['Data/Regions files/Latin America.csv',
 'Data/Regions files/Developed Markets.csv',
 'Data/Regions files/Frontier Markets.csv',
 'Data/Regions files/Developed Europe.csv',
 'Data/Regions files/Emerging Europe.csv',
 'Data/Regions files/Developed Asia Pacific.csv',
 'Data/Regions files/Middle East.csv',
 'Data/Regions files/North America.csv',
 'Data/Regions files/Emerging Markets.csv',
 'Data/Regions files/Emerging Asia Pacific.csv',
 'Data/Regions files/Africa.csv',
 'Data/Regions files/Global.csv',
 'Data/Regions files/Broad Asia.csv',
 'Data/Regions files/Global ex-U.S..csv']

In [175]:
regions_dict = {}
for file in filelist:
    region = file.split("/")[-1].strip(".csv")
    data = pd.read_csv(file)
    symbols = list(data["Symbol"])
    
    # regions_dict.update({region:symbols})
    for sym in symbols:     
        regions_dict.update({sym:{"region":region}})
        

In [179]:
with open("Data/ETFdata_etfdb_regions.json", "w") as outfile:
    # Write to json file
    json.dump(regions_dict, outfile)